# CPSC 330 - Applied Machine Learning 

## Homework 7: Word embeddings and topic modeling 
**Due date: [Apr 2, 11:59pm](https://github.com/UBC-CS/cpsc330-2023W2?tab=readme-ov-file#deliverable-due-dates-tentative).**

## Imports

In [1]:
import os
from hashlib import sha1

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline

<br><br>

## Submission instructions
<hr>

_points: 2_

You will receive marks for correctly submitting this assignment. To submit this assignment, follow the instructions below:

- **You may work on this assignment in a group (group size <= 4) and submit your assignment as a group.** 
- Below are some instructions on working as a group.  
    - The maximum group size is 4. 
    - You can choose your own group members. 
    - Use group work as an opportunity to collaborate and learn new things from each other. 
    - Be respectful to each other and make sure you understand all the concepts in the assignment well. 
    - It's your responsibility to make sure that the assignment is submitted by one of the group members before the deadline. [Here](https://help.gradescope.com/article/m5qz2xsnjy-student-add-group-members) are some instructions on adding group members in Gradescope.  
- Be sure to follow the [homework submission instructions](https://github.com/UBC-CS/cpsc330-2023W2/blob/main/docs/homework_instructions.md).
- Upload the .ipynb file to PrairieLearn.

<br><br><br><br>

## Exercise 1:  Exploring pre-trained word embeddings <a name="1"></a>
<hr>

In lecture 17, we talked about natural language processing (NLP). Using pre-trained word embeddings is very common in NLP. It has been shown that pre-trained word embeddings work well on a variety of text classification tasks. These embeddings are created by training a model like Word2Vec on a huge corpus of text such as a dump of Wikipedia or a dump of the web crawl. 

A number of pre-trained word embeddings are available out there. Some popular ones are: 

- [GloVe](https://nlp.stanford.edu/projects/glove/)
    * trained using [the GloVe algorithm](https://nlp.stanford.edu/pubs/glove.pdf) 
    * published by Stanford University 
- [fastText pre-trained embeddings for 294 languages](https://fasttext.cc/docs/en/pretrained-vectors.html) 
    * trained using the fastText algorithm
    * published by Facebook
    
In this exercise, you will be exploring GloVe Wikipedia pre-trained embeddings. The code below loads the word vectors trained on Wikipedia using an algorithm called Glove. You'll need `gensim` package in your cpsc330 conda environment to run the code below. 

```
> conda activate cpsc330
> conda install -c anaconda gensim
```

In [2]:
import gensim
import gensim.downloader

print(list(gensim.downloader.info()["models"].keys()))

['fasttext-wiki-news-subwords-300', 'conceptnet-numberbatch-17-06-300', 'word2vec-ruscorpora-300', 'word2vec-google-news-300', 'glove-wiki-gigaword-50', 'glove-wiki-gigaword-100', 'glove-wiki-gigaword-200', 'glove-wiki-gigaword-300', 'glove-twitter-25', 'glove-twitter-50', 'glove-twitter-100', 'glove-twitter-200', '__testing_word2vec-matrix-synopsis']


In [3]:
# This will take a while to run when you run it for the first time.
import gensim.downloader as api

glove_wiki_vectors = api.load("glove-wiki-gigaword-100")

In [4]:
len(glove_wiki_vectors)

400000

There are 400,000 word vectors in this pre-trained model. 

Now that we have GloVe Wiki vectors loaded in `glove_wiki_vectors`, let's explore the embeddings. 

<br><br>

<!-- BEGIN QUESTION -->

### 1.1 Word similarity using pre-trained embeddings

_Points:_ 2

**Your tasks:**

- Come up with a list of 4 words of your choice and find similar words to these words in `glove_wiki_vectors` embeddings. 

<div class="alert alert-warning">

Solution_1.1
    
</div>

In [5]:
glove_wiki_vectors.most_similar("amazon")

[('amazon.com', 0.6525673866271973),
 ('ebay', 0.6066174507141113),
 ('bezos', 0.5970092415809631),
 ('rainforest', 0.5607468485832214),
 ('kindle', 0.5589122772216797),
 ('netscape', 0.549601137638092),
 ('yahoo', 0.5490650534629822),
 ('google', 0.5455811023712158),
 ('itunes', 0.5371230840682983),
 ('users', 0.5195276141166687)]

In [6]:
glove_wiki_vectors.most_similar("job")

[('jobs', 0.7932984232902527),
 ('better', 0.7354214191436768),
 ('doing', 0.7352380156517029),
 ('working', 0.7307748198509216),
 ('work', 0.7281291484832764),
 ('hiring', 0.7160316705703735),
 ('good', 0.7088245153427124),
 ('done', 0.7070804834365845),
 ('going', 0.7031288146972656),
 ('now', 0.6979781985282898)]

In [7]:
glove_wiki_vectors.most_similar("offer")

[('offered', 0.905266523361206),
 ('offering', 0.8891221880912781),
 ('offers', 0.8301975727081299),
 ('give', 0.7609235644340515),
 ('provide', 0.7608141899108887),
 ('buy', 0.738306999206543),
 ('accept', 0.7360981106758118),
 ('receive', 0.7277234792709351),
 ('pay', 0.7245213389396667),
 ('agreed', 0.7243043184280396)]

In [8]:
glove_wiki_vectors.most_similar("please")

[('ask', 0.7292373776435852),
 ('tell', 0.715707004070282),
 ('you', 0.6839258074760437),
 ('wish', 0.6618576645851135),
 ('call', 0.6535139679908752),
 ('forget', 0.651892364025116),
 ('your', 0.6507151126861572),
 ('let', 0.6474228501319885),
 ("'ll", 0.6414669156074524),
 ('write', 0.6338616013526917)]

<!-- END QUESTION -->

<br><br>

### 1.2 Word similarity using pre-trained embeddings

_Points:_ 2

**Your tasks:**
1. Calculate cosine similarity for the following word pairs (`word_pairs`) using the [`similarity`](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) method of `glove_wiki_vectors`.

In [9]:
word_pairs = [
    ("coast", "shore"),
    ("clothes", "closet"),
    ("old", "new"),
    ("smart", "intelligent"),
    ("dog", "cat"),
    ("tree", "lawyer"),
]

<div class="alert alert-warning">

Solution_1.2
    
</div>

In [10]:
coast_shore_similarity = glove_wiki_vectors.similarity(word_pairs[0][0], word_pairs[0][1])
clothes_closet_similarity = glove_wiki_vectors.similarity(word_pairs[1][0], word_pairs[1][1])
old_new_similarity = glove_wiki_vectors.similarity(word_pairs[2][0], word_pairs[2][1])
smart_intelligent_similarity = glove_wiki_vectors.similarity(word_pairs[3][0], word_pairs[3][1])
dog_cat_similarity = glove_wiki_vectors.similarity(word_pairs[4][0], word_pairs[4][1])
tree_lawyer_similarity = glove_wiki_vectors.similarity(word_pairs[5][0], word_pairs[5][1])

similarities = [coast_shore_similarity, clothes_closet_similarity, old_new_similarity, smart_intelligent_similarity, dog_cat_similarity, tree_lawyer_similarity]
similarities

[0.7000272, 0.546276, 0.6432488, 0.7552732, 0.8798075, 0.076719455]

<br><br>

<!-- BEGIN QUESTION -->

### 1.3 Stereotypes and biases in embeddings

_Points:_ 6

Word vectors contain lots of useful information. But they also contain stereotypes and biases of the texts they were trained on. In the lecture, we saw an example of gender bias in Google News word embeddings. Here we are using pre-trained embeddings trained on Wikipedia data. 

**Your tasks:**

1. Explore whether there are any worrisome biases or stereotypes present in these embeddings or not by trying out at least 4 examples. You can use the following two methods or other methods of your choice to explore what kind of stereotypes and biases are encoded in these embeddings. 
    - the `analogy` function below which gives word analogies (an example shown below)
    - [similarity](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=similarity#gensim.models.keyedvectors.KeyedVectors.similarity) or [distance](https://radimrehurek.com/gensim/models/keyedvectors.html?highlight=distance#gensim.models.keyedvectors.KeyedVectors.distances) methods (an example is shown below)   
2. Discuss your observations.

> Note that most of the recent embeddings are de-biased. But you might still observe some biases in them. Also, not all stereotypes present in pre-trained embeddings are necessarily bad. But you should be aware of them when you use them in your models. 

An example of using analogy to explore biases and stereotypes.

In [11]:
def analogy(word1, word2, word3, model=glove_wiki_vectors):
    """
    Returns analogy word using the given model.

    Parameters
    --------------
    word1 : (str)
        word1 in the analogy relation
    word2 : (str)
        word2 in the analogy relation
    word3 : (str)
        word3 in the analogy relation
    model :
        word embedding model

    Returns
    ---------------
        pd.dataframe
    """
    print("%s : %s :: %s : ?" % (word1, word2, word3))
    sim_words = model.most_similar(positive=[word3, word2], negative=[word1])
    return pd.DataFrame(sim_words, columns=["Analogy word", "Score"])

In [12]:
analogy("man", "doctor", "woman")

man : doctor :: woman : ?


,Analogy word,Score
0,nurse,0.773523
1,physician,0.718943
2,doctors,0.682433
3,patient,0.675068
4,dentist,0.672603
5,pregnant,0.664246
6,medical,0.652045
7,nursing,0.645348
8,mother,0.639333
9,hospital,0.638750


An example of using similarity between words to explore biases and stereotypes.  

In [13]:
glove_wiki_vectors.similarity("white", "rich")

0.447236

In [14]:
glove_wiki_vectors.similarity("black", "rich")

0.5174519

<div class="alert alert-warning">

Solution_1.3
    
</div>

In [15]:
analogy("black", "basketball", "asian")

black : basketball :: asian : ?


,Analogy word,Score
0,volleyball,0.665086
1,badminton,0.647040
2,soccer,0.636133
3,cba,0.635359
4,tournament,0.633246
5,championships,0.618375
6,hockey,0.612923
7,finals,0.602023
8,weightlifting,0.600914
9,nba,0.590800


In [16]:
analogy("white", "potato", "chinese")

white : potato :: chinese : ?


,Analogy word,Score
0,dumplings,0.579126
1,noodle,0.553679
2,vegetable,0.523076
3,poultry,0.522655
4,chicken,0.504606
5,dairy,0.498520
6,noodles,0.496295
7,xiangqi,0.495747
8,sushi,0.493907
9,dishes,0.492692


In [17]:
analogy("black", "hip-hop", "korean")

black : hip-hop :: korean : ?


,Analogy word,Score
0,dance-pop,0.553936
1,k-pop,0.547137
2,dprk,0.529534
3,r&b,0.506273
4,butoh,0.459998
5,gaon,0.453590
6,svensktoppen,0.453092
7,korea,0.452844
8,chart,0.450302
9,charting,0.445989


In [18]:
analogy("man", "job", "woman")

man : job :: woman : ?


,Analogy word,Score
0,jobs,0.720464
1,employment,0.651589
2,care,0.643972
3,hiring,0.638583
4,nursing,0.617894
5,working,0.605381
6,she,0.602907
7,applicants,0.584693
8,doing,0.578530
9,staff,0.577949


There's definitely some examples of racial biases in these analogies, but it's significantly less biased than what was expected, probably explained by the fact the recent embeddings are de-biased. In one example, analogy didn't output a word conceptually different from the comparison word2 (like job and jobs). In the remaining, the analogies may be considered racially biased but the scores are very low which shows that there isn't a strong bias.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 1.4 Discussion

_Points:_ 4

**Your tasks:**
1. Based on your exploration above, comment on the overall quality of these pre-trained embeddings. 
2. In the lecture, we saw that our pre-trained word embedding model output an analogy that reinforced a gender stereotype. Give an example of how using such a model could cause harm in the real world.

<div class="alert alert-warning">

Solution_1.4
    
</div>

1. Based on the previous exploration, the embeddings avoided significant racial and sexual biased answers, but this sometimes resulted in analogies that weren't different at all from the word to find an analogy for (like jobs for job). If de-biased results are the priority, then the embeddings are of fairly good quality.

2. Using the biased in-class example (man-computer_programer, woman-homemaker), there could be potential negative repercussions in the real world. For example, if this model was used to recommend job openings for new applicants, a female applicant might get a collection of recommendations skewed towards the stereotypes listed. A female PhD student might get school teacher recommendations or a medical student might get registered nurse recommendations.

<!-- END QUESTION -->

<br><br>

### 1.5 Classification with pre-trained embeddings 

_Points:_ 8

In lecture 17, we saw that you can conveniently get word vectors with `spaCy` with `en_core_web_md` model. In this exercise, you'll use word embeddings in multi-class text classification task. We will use [HappyDB](https://www.kaggle.com/ritresearch/happydb) corpus which contains about 100,000 happy moments classified into 7 categories: *affection, exercise, bonding, nature, leisure, achievement, enjoy_the_moment*. The data was crowd-sourced via [Amazon Mechanical Turk](https://www.mturk.com/). The ground truth label is not available for all examples, and in this lab, we'll only use the examples where ground truth is available (~15,000 examples). 

- Download the data from [here](https://www.kaggle.com/ritresearch/happydb).
- Unzip the file and copy it under the data/ directory in this homework directory.

The code below reads the data CSV (assuming that it's present in the current directory as *cleaned_hm.csv*),  cleans it up a bit, and splits it into train and test splits. 

**Your tasks:**

1. Train logistic regression with bag-of-words features (`CountVectorizer`, be sure to filter out stop words), calculate the train and test accuracy and show a [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.
2. Train logistic regression with average embedding representation extracted using spaCy, calculate the train and test accuracy and show the [classification report](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.classification_report.html) on the test set.

In [19]:
df = pd.read_csv("data/cleaned_hm.csv", index_col=0)
sample_df = df.dropna()
sample_df.head()

,wid,reflection_period,original_hm,cleaned_hm,modified,num_sentence,ground_truth_category,predicted_category
hmid,,,,,,,,
27676,206,24h,We had a serious talk with some friends of our...,We had a serious talk with some friends of our...,True,2,bonding,bonding
27678,45,24h,I meditated last night.,I meditated last night.,True,1,leisure,leisure
27697,498,24h,My grandmother start to walk from the bed afte...,My grandmother start to walk from the bed afte...,True,1,affection,affection
27705,5732,24h,I picked my daughter up from the airport and w...,I picked my daughter up from the airport and w...,True,1,bonding,affection
27715,2272,24h,when i received flowers from my best friend,when i received flowers from my best friend,True,1,bonding,bonding


In [20]:
sample_df = sample_df.rename(
    columns={"cleaned_hm": "moment", "ground_truth_category": "target"}
)

In [21]:
train_df, test_df = train_test_split(sample_df, test_size=0.3, random_state=123)
X_train, y_train = train_df["moment"], train_df["target"]
X_test, y_test = test_df["moment"], test_df["target"]

You need `spacy` to run the code below. If it's not in your course conda environment, you need to install it. 

> conda install -c conda-forge spacy

You also need to download the following language model. 

> python -m spacy download en_core_web_md

In [22]:
import spacy
import en_core_web_md

# nlp = spacy.load("en_core_web_md")
nlp = en_core_web_md.load()

<div class="alert alert-warning">

Solution_1.5
    
</div>

In [23]:
from sklearn.metrics import classification_report

vec_bow = CountVectorizer(stop_words="english")
lr_bow = LogisticRegression(max_iter=2000)
pipe_bow = make_pipeline(vec_bow, lr_bow)

pipe_bow.fit(X_train, y_train)

bow_train_accuracy = pipe_bow.score(X_train, y_train)
bow_test_accuracy = pipe_bow.score(X_test, y_test)
bow_report = classification_report(y_test, pipe_bow.predict(X_test)) # use digits=2

print(f"Train Accuracy: {bow_train_accuracy}\n")
print(f"Test Accuracy: {bow_test_accuracy}\n")
print(bow_report)

Train Accuracy: 0.9562051178314959

Test Accuracy: 0.8173666823973572

                  precision    recall  f1-score   support

     achievement       0.79      0.87      0.83      1302
       affection       0.90      0.91      0.91      1423
         bonding       0.91      0.85      0.88       492
enjoy_the_moment       0.60      0.54      0.57       469
        exercise       0.91      0.57      0.70        74
         leisure       0.73      0.70      0.71       407
          nature       0.73      0.46      0.57        71

        accuracy                           0.82      4238
       macro avg       0.80      0.70      0.74      4238
    weighted avg       0.82      0.82      0.81      4238



In [24]:
X_train_embeddings = []
for doc in nlp.pipe(X_train):
  X_train_embeddings.append(doc.vector)

X_test_embeddings = []
for doc in nlp.pipe(X_test):
  X_test_embeddings.append(doc.vector)

In [25]:
lr_emb = LogisticRegression(max_iter=2000)
lr_emb.fit(X_train_embeddings, y_train)

c:\Users\russe\miniconda3\envs\cpsc330\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(max_iter=2000)

In [26]:
emb_train_accuracy = lr_emb.score(X_train_embeddings, y_train)
emb_test_accuracy = lr_emb.score(X_test_embeddings, y_test)
emb_report = classification_report(y_test, lr_emb.predict(X_test_embeddings)) # use digits=2

print(f"Train Accuracy: {emb_train_accuracy}\n")
print(f"Test Accuracy: {emb_test_accuracy}\n")
print(emb_report)

Train Accuracy: 0.8494993425710529

Test Accuracy: 0.7928268050967437

                  precision    recall  f1-score   support

     achievement       0.81      0.83      0.82      1302
       affection       0.86      0.91      0.89      1423
         bonding       0.83      0.77      0.80       492
enjoy_the_moment       0.56      0.51      0.53       469
        exercise       0.66      0.73      0.69        74
         leisure       0.72      0.65      0.68       407
          nature       0.67      0.72      0.69        71

        accuracy                           0.79      4238
       macro avg       0.73      0.73      0.73      4238
    weighted avg       0.79      0.79      0.79      4238



<br><br>

<!-- BEGIN QUESTION -->

### 1.6 Discussion

_Points:_ 6

**Your tasks:**
1. Briefly explain the difference between using `CountVectorizer` vs. average-embedding approach for text classification.  
2. Which representation among these two would be more interpretable? Why?   
3. Are we using any transfer learning here? If yes, are you observing any benefits of transfer learning? Briefly discuss. 

<div class="alert alert-warning">

Solution_1.6
    
</div>

1. CountVectorizer quantifies word occurrence or existence without taking context or semantics into consideration. Meanwhile, average-embedding calculates the average of word embedding vecors which take text semantics and context into consideration

2. CountVectorizer transformations can be very dense or sparse depending on the parameters set, but is considered more interpretable regardless due to the simplicity of the transformation which only cares about individual word occurrence

3. Our average-embedding approach utilizes the word embeddings from `spaCy`. We are employing transfer learning since `spaCy` pre-trains word embeddings from various text data. We benefit from this by not having to train from scratch allowing for runtime savings.

<!-- END QUESTION -->

<br><br><br><br>

## Exercise 2: Topic modeling 

The overarching goal of topic modeling is understanding high-level themes in a large collection of texts in an unsupervised way. 

In this exercise you will explore topics in a subset of `scikit-learn`'s [20 newsgroups text dataset](https://scikit-learn.org/0.19/datasets/twenty_newsgroups.html) using `scikit-learn`'s `LatentDirichletAllocation` (LDA) model. 

Usually, topic modeling is used for discovering abstract "topics" that occur in a collection of documents when you do not know the actual topics present in the documents. But 20 newsgroups text dataset is labeled with categories (e.g., sports, hardware, religion), and you will be able to cross-check the topics discovered by your model with these available topics. 

The starter code below loads the train and test portion of the data and convert the train portion into a pandas DataFrame. For speed, we will only consider documents with the following 8 categories. 

In [27]:
from sklearn.datasets import fetch_20newsgroups

In [28]:
cats = [
    "rec.sport.hockey",
    "rec.sport.baseball",
    "soc.religion.christian",
    "alt.atheism",
    "comp.graphics",
    "comp.windows.x",
    "talk.politics.mideast",
    "talk.politics.guns",
]  # We'll only consider these categories out of 20 categories for speed.

newsgroups_train = fetch_20newsgroups(
    subset="train", remove=("headers", "footers", "quotes"), categories=cats
)
X_news_train, y_news_train = newsgroups_train.data, newsgroups_train.target
df = pd.DataFrame(X_news_train, columns=["text"])
df["target"] = y_news_train
df["target_name"] = [
    newsgroups_train.target_names[target] for target in newsgroups_train.target
]
df

,text,target,target_name
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics
3,"The following problem is really bugging me,\na...",2,comp.windows.x
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast
...,...,...,...
4558,Hi Everyone ::\n\nI am looking for some soft...,1,comp.graphics
4559,Archive-name: x-faq/part3\nLast-modified: 1993...,2,comp.windows.x
4560,"\nThat's nice, but it doesn't answer the quest...",6,talk.politics.guns
4561,"Hi,\n I just got myself a Gateway 4DX-33V ...",2,comp.windows.x


In [29]:
newsgroups_train.target_names

['alt.atheism',
 'comp.graphics',
 'comp.windows.x',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast']

<br><br>

<!-- BEGIN QUESTION -->

### 2.1 Preprocessing using [spaCy](https://spacy.io/)

_Points:_ 8

In this exercise you'll prepare the data for topic modeling using [spaCy](https://spacy.io/). Preprocessing is a crucial step before training an LDA model and it markedly affects topic modeling results. So let's carry out preprocessing. 

**Your tasks:** 

Write code to carry out preprocessing of the "text" column in the dataframe above and store the preprocessed text in a new column called "text_pp" in the dataframe. 


Note that there is no such thing as "perfect" preprocessing. You'll have to make your own judgments and decisions on which tokens are more informative and which ones are less informative for the given task. Some common text preprocessing steps for topic modeling are: 
- getting rid of slashes or other weird characters
- sentence segmentation and tokenization      
- getting rid of urls and email addresses
- getting rid of other fairly unique tokens which are not going to help us in topic modeling  
- excluding stopwords and punctuation 
- lemmatization        

You might have to go back and forth between the preprocessing and topic modeling and interpretation steps in the next exercises. 

> Check out [these available attributes](https://spacy.io/api/token#attributes) for `token` in spaCy which might help you with preprocessing. 

> You can also get rid of words with specific POS tags. [Here](https://spacy.io/api/annotation/#pos-en) is the list of part-of-speech tags used in spaCy. 

> Note that preprocessing the corpus might take some time. So here are a couple of suggestions: 1) During the debugging phase, work on a smaller subset of the data. 2) Once you're done with the preprocessing part, you might want to save the preprocessed data so that you don't run the preprocessing part every time you run the notebook. 


<div class="alert alert-warning">

Solution_2_1
    
</div>

Preprocess code was inspired by [Lecture 17 notes](https://github.com/UBC-CS/cpsc330-2023W2/blob/5a6fd0c9be17b08752a5e8d3f003168fae0569ea/lectures/mathias/17_natural-language-processing.ipynb)

In [30]:
def preprocess(
    doc,
    min_token_len=2,
    irrelevant_pos=["ADV", "PRON", "CCONJ", "PUNCT", "PART", "DET", "ADP", "SPACE", "NUM"]
):
    clean_text = []
    for token in doc:
        if (
            token.is_stop == False
            and len(token) > min_token_len
            and token.pos_ not in irrelevant_pos
            and token.like_url == False
            and token.like_email == False
        ):
            lemma = token.lemma_
            clean_text.append(lemma.lower())
    return " ".join(clean_text)

In [31]:
df["text_pp"] = [preprocess(text) for text in nlp.pipe(df["text"])]

In [32]:
df.loc[df["target_name"] == "talk.politics.guns"]

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. sence wonder help u.s.c. prov...
10,\n\n\nTwo questions:\n\n1) You asserted that ...,6,talk.politics.guns,question assert ar-15 ak-47 suitable real hunt...
14,\nHmm. I beg to differ. It will probably mak...,6,talk.politics.guns,hmm beg differ big difference point true major...
33,\nNational Rifle Association\n1600 Rhode Islan...,6,talk.politics.guns,national rifle association rhode island ave wa...
66,"\nNope. Here in Northern California, a newspa...",6,talk.politics.guns,nope northern california newspaper survey ask ...
...,...,...,...,...
4539,"\nUnfortunately, elections can, and are, bough...",6,talk.politics.guns,election buy promise voter money vote convict ...
4552,On the subject of CS/CN/tear gas: when I recei...,6,talk.politics.guns,subject tear gas receive initial introduction ...
4553,"}>Look, if you can figure out a reliable means...",6,talk.politics.guns,look figure reliable mean keep gun bad people ...
4557,"According to WNCI 97.9 FM radio this morning, ...",6,talk.politics.guns,accord wnci radio morning dayton ohio operate ...


<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.2 Justification

_Points:_ 2

**Your tasks:**

Outline the preprocessing steps you carried out in the previous exercise and provide a brief justification for these steps.

<div class="alert alert-warning">

Solution_2_2
    
</div>

The steps followed were based on what was recommended in the instructions. We created a preprocess method that filtered out tokens based on certain criteria:
1. Filter out stop tokens
2. Filter out short tokens (< 2 length)
3. Filter out certain tokens (punctuation, spaces, etc.)
4. Filter out urls and emails

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.3 Build a topic model using sklearn's LatentDirichletAllocation

_Points:_ 4

**Your tasks:**
1. Create a topic model on the preprocessed data using [sklearn's `LatentDirichletAllocation`](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.LatentDirichletAllocation.html). Pick a reasonable number for `n_components`, i.e., number of topics and briefly justify your choice.

<div class="alert alert-warning">

Solution_2_3
    
</div>

In [33]:
vec_lda = CountVectorizer(stop_words="english")
text_pp_bow = vec_lda.fit_transform(df["text_pp"])

In [34]:
from sklearn.decomposition import LatentDirichletAllocation

n_topics = 8 # number of topics
lda = LatentDirichletAllocation(
    n_components=n_topics, learning_method="batch", max_iter=10, random_state=42
)
lda.fit(text_pp_bow) 
document_topics = lda.transform(text_pp_bow)

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.4 Exploring word topic association

_Points:_ 5

**Your tasks:**
1. Show top 10 words for each of your topics and suggest labels for each of the topics (similar to how we came up with labels "health and nutrition", "fashion", and "machine learning" in the toy example we saw in class). 

> If your topics do not make much sense, you might have to go back to preprocessing in Exercise 2.1, improve it, and train your LDA model again. 

<div class="alert alert-warning">

Solution_2_4
    
</div>

Code below was inspired by [Lecture 17 notes](https://github.com/UBC-CS/cpsc330-2023W2/blob/5a6fd0c9be17b08752a5e8d3f003168fae0569ea/lectures/mathias/17_natural-language-processing.ipynb)

In [35]:
sorting = np.argsort(lda.components_, axis=1)[:, ::-1]
feature_names = np.array(vec_lda.get_feature_names_out())

In [36]:
import mglearn

mglearn.tools.print_topics(
    topics=range(n_topics),
    feature_names=feature_names,
    sorting=sorting,
    topics_per_chunk=5,
    n_words=10,
)

topic 0       topic 1       topic 2       topic 3       topic 4       
--------      --------      --------      --------      --------      
game          people        period        god           gun           
team          armenian      play          people        entry         
year          israel        team          think         think         
play          government    nhl           believe       use           
good          turkish       new           know          time          
player        jews          goal          religion      firearm       
season        right         pts           question      law           
win           armenians     hockey        thing         weapon        
think         state         power         bible         people        
time          say           pittsburgh    atheist       know          


topic 5       topic 6       topic 7       
--------      --------      --------      
know          god           file          
say           jes

Based on the topic words and selecting from the categories we outlined earlier, we believe the following describes the topics correctly:

0. Sports
1. Mideast
2. Hockey
3. Atheism
4. Guns
5. Other???
6. Christianity
7. Windows Operating System

Topics 0 and 5 and relatively arbitrary, but based on elimination, topic 0 most likely matches `Baseball` and topic 5 likely matches `Graphics`. After multiple reconfigurations of the preprocessing method, this was the best grouping outcome that was realized.

<!-- END QUESTION -->

<br><br>

<!-- BEGIN QUESTION -->

### 2.5 Exploring document topic association

_Points:_ 5

**Your tasks:**
1. Show the document topic assignment of the first five documents from `df`. 
2. Comment on the document topic assignment of the model. 

<div class="alert alert-warning">

Solution_2_5
    
</div>

In [37]:
df.loc[0:4]

,text,target,target_name,text_pp
0,"You know, I was reading 18 U.S.C. 922 and some...",6,talk.politics.guns,know read u.s.c. sence wonder help u.s.c. prov...
1,\n\n\nIt's not a bad question: I don't have an...,1,comp.graphics,bad question ref list algorithm think bit hard...
2,"\nActuallay I don't, but on the other hand I d...",1,comp.graphics,actuallay hand support idea have newsgroup asp...
3,"The following problem is really bugging me,\na...",2,comp.windows.x,follow problem bug appreciate help create wind...
4,\n\n This is the latest from UPI \n\n For...,7,talk.politics.mideast,late upi foreign ministry spokesman ferhat ata...


In [38]:
document_topics[0:5]

array([[0.00245361, 0.37201818, 0.00245121, 0.00245444, 0.4780812 ,
        0.00245534, 0.05486239, 0.08522363],
       [0.23711771, 0.00176271, 0.00176133, 0.00176396, 0.00176272,
        0.3041309 , 0.00176252, 0.44993815],
       [0.00240686, 0.23403283, 0.00240523, 0.49215527, 0.00240765,
        0.00240831, 0.00240678, 0.26177707],
       [0.00431166, 0.00431475, 0.00431061, 0.00431391, 0.51905312,
        0.00431788, 0.00431486, 0.4550632 ],
       [0.00290865, 0.77611419, 0.00290739, 0.00290856, 0.00290857,
        0.00290921, 0.20643382, 0.00290959]])

Based on earlier topic classifications, the first five documents are made up in the following ways:

1. 48% Guns, 37% Mideast, 9% Windows Operating System, 5% Christianity
2. 45% Windows Operating System, 30% Graphics, 24% Baseball
3. 49% Atheism, 26% Windows Operating System, 23% Mideast
4. 52% Guns, 46% Windows Operating System
5. 78% Mideast, 21% Christianity

Documents 1 and 5's real categories are correctly identified based on the most populated topics. Document 2 and 4 are almost the same with their respective categories being the 2nd most populated topics. Document 3 is the least accurate, but it's worth noticing that the 2nd most populated topic `Windows Operating System` has plenty of similarities with the `Graphics` topic.

<!-- END QUESTION -->

<br><br><br><br>

<!-- BEGIN QUESTION -->

## Exercise 3: Short answer questions 
<hr>

_Points:_ 6

1. In lecture 18, we talked about multi-class classification. Comment on how each model in the list below might be handling multiclass classification. Check `scikit-learn` documentation for each of these models when you answer this question.  
    - Decision Tree
    - KNN
    - Random Forest    
    - Logistic Regression
    - SVM RBF
2. What is transfer learning in natural language processing or computer vision? Briefly explain.     
3. In Lecture 18 we briefly discussed how neural networks are sort of like `Pipeline`s, in the sense that they involve multiple sequential transformations of the data, finally resulting in the prediction. Why was this property useful when it came to transfer learning?

<div class="alert alert-warning">

Solution_3
    
</div>

1. Handling multiclass classification based on model:
- Decision Tree: This model is able to naturally handle multiclass classification. It does this by predicting the value of a target class by learning simple decision rules from the data features.
- KNN: This model is capable of handling multiclass classification. It does this by finding the k-nearest neighbors based on a selected data point usually based on Euclidean distance (in this course). Then a class is assigned based on the majority class of the k-nearest neighbors.
- Random Forest: This model is able to handle multiclass classification by default and extends the Decision Tree model. This model is able to do this by collecting the predictions of multiple decision trees and then determining the class based on the majority.
- Logistic Regression: The model can be extended for multiclass classification however it will require techniques such as the one-versus-rest reduction as stated scikit-learn documentation.
- SVM RBF: This model can be extended for multiclass classification however it requires implementing the one-versus-one approach which it will then be transformed by the one-vs-rest decision function in order to provide a consistent interface with other classifiers.

2. Transfer learning is a strategy where a model trained on one task is then repurposed to perform another similar task. In natural language processing or computer vision, this refers to training a model on a large dataset for a specific task, then the model is fine-tuned on a smaller dataset for a different but similar task. This knowledge transfer from tasks enables better performance even with limited labeled data.

3. Neural networks' property of performing multiple sequential transformations of data is useful in transfer learning. This is because in transfer learning, a pre-trained neural network which learned general features from a large dataset can be fine-tuned on a smaller dataset for a related task. The initial layers of the network, which extract basic features are useful for general tasks, while the layers after are fine-tuned to task-specific features. This property overall helps the model to effectively leverage knowledge from the source task thus improving performance on the target task with limited data.
  

<!-- END QUESTION -->

<br><br><br><br>

**PLEASE READ BEFORE YOU SUBMIT:** When you are ready to submit your assignment do the following:

1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Clear All Outputs` and then `Run -> Run All Cells`. 
2. Notebooks with cell execution numbers out of order or not starting from “1” will have marks deducted. Notebooks without the output displayed may not be graded at all (because we need to see the output in order to grade your work).
3. Upload the assignment using [PrairieLearn](https://ca.prairielearn.com/pl/course_instance/6697). Don't forget to rename your file `hw6_sol.ipynb`.

![](img/eva-well-done.png)